In [1]:
#THERE ARE TWO SEPARATE TARGET COLUMNS TO PREDICT, EITHER IN DATASET 1 AND 2

In [2]:
#import libraries
import numpy as np
import pandas as pd
import os

In [3]:
#import datasets
df1 = pd.read_csv("../data/dataset_1.csv")

In [4]:
# dataset 1 info
print(df1.shape)
df1.head()

(50000, 301)


,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300,target
0,0,0,0.0,0.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
1,0,0,0.0,3.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
2,0,0,0.0,5.88,0.0,0,0,0,0,0,...,0.0,0,0,3,0,0,0,0.0,67772.7216,0
3,0,0,0.0,14.10,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
4,0,0,0.0,5.76,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0


In [5]:
substr = "tar"
for s in df1.columns:
    if substr in s:
        print(f"{s} contains {substr}")

target contains tar


In [6]:
# dataset 2 info
print(df1.shape)
df1.head()

(50000, 301)


,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_292,var_293,var_294,var_295,var_296,var_297,var_298,var_299,var_300,target
0,0,0,0.0,0.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
1,0,0,0.0,3.00,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
2,0,0,0.0,5.88,0.0,0,0,0,0,0,...,0.0,0,0,3,0,0,0,0.0,67772.7216,0
3,0,0,0.0,14.10,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0
4,0,0,0.0,5.76,0.0,0,0,0,0,0,...,0.0,0,0,0,0,0,0,0.0,0.0000,0


In [7]:
# split merged dataset into (train/test, validate) = (80%, 20%)
df1, val1 = np.split(df1.sample(frac=1, random_state=42), [int(.8*len(df1))])

In [8]:
# Create a relative path to the directory you want to create
relative_path = "./validation"

# Use os.makedirs to create the directory if it does not exist
if not os.path.exists(relative_path):
    os.makedirs(relative_path)

# exporting validation dataset to csv
val1.to_csv('validation/val1.csv', encoding='utf-8')

In [9]:
# showing dublicate columns in data frame
from fast_ml.utilities import display_all
from fast_ml.feature_selection import get_duplicate_features

# checks weather there are duplicate columns names
duplicates = list(set([x for x in list(df1.columns) if list(df1.columns).count(x) > 1]))
print(len(duplicates))

duplicate_features = get_duplicate_features(df1)
duplicate_features.head(10)

0


,Desc,feature1,feature2
0,Duplicate Values,var_23,var_196
1,Duplicate Values,var_23,var_113
2,Duplicate Values,var_23,var_122
3,Duplicate Values,var_23,var_135
4,Duplicate Values,var_23,var_158
5,Duplicate Values,var_23,var_167
6,Duplicate Values,var_23,var_171
7,Duplicate Values,var_89,var_178
8,Duplicate Values,var_89,var_180
9,Duplicate Values,var_23,var_182


In [10]:
# flaten dataframe with duplicate columns to list
duplicate_features = duplicate_features.query("Desc=='Duplicate Values'")['feature2'].to_list()
print(duplicate_features)

['var_196', 'var_113', 'var_122', 'var_135', 'var_158', 'var_167', 'var_171', 'var_178', 'var_180', 'var_182', 'var_195', 'var_289', 'var_201', 'var_212', 'var_215', 'var_225', 'var_223', 'var_248', 'var_294', 'var_297', 'var_183', 'var_104', 'var_287', 'var_148', 'var_106', 'var_227', 'var_216', 'var_120', 'var_127', 'var_112', 'var_296', 'var_151', 'var_269', 'var_116', 'var_232', 'var_263', 'var_285', 'var_99', 'var_199', 'var_239', 'var_250', 'var_33', 'var_81', 'var_97', 'var_92', 'var_44', 'var_87', 'var_80', 'var_61', 'var_69']


In [11]:
# drops duplicate columns and shows difference to starting df
print('Shape of Dataset before dropping the duplicate values features: ', df1.shape)
df1.drop(columns = duplicate_features, inplace=True)
print('Shape of Dataset after dropping the duplicate values features: ', df1.shape)

Shape of Dataset before dropping the duplicate values features:  (40000, 301)
Shape of Dataset after dropping the duplicate values features:  (40000, 251)


In [12]:
from fast_ml.feature_selection import get_constant_features

constant_features = get_constant_features(df1)
constant_features.head(10)

,Desc,Var,Value,Perc
0,Constant,var_89,0.0,100.0000
1,Constant,var_23,0.0,100.0000
2,Quasi Constant,var_65,0.0,99.9975
3,Quasi Constant,var_247,0.0,99.9975
4,Quasi Constant,var_34,0.0,99.9975
5,Quasi Constant,var_124,0.0,99.9975
6,Quasi Constant,var_36,0.0,99.9975
7,Quasi Constant,var_66,0.0,99.9975
8,Quasi Constant,var_129,0.0,99.9975
9,Quasi Constant,var_280,0.0,99.9975


In [13]:
print("first row data = \n", constant_features.iloc[0], "\n number of rows = \n", len(constant_features))

first row data = 
 Desc     Constant
Var        var_89
Value         0.0
Perc        100.0
Name: 0, dtype: object 
 number of rows = 
 132


In [14]:
constant_features = constant_features.loc[(constant_features.Perc > 0.99)]
constant_features.head(10)
print(constant_features.shape)

(132, 4)


In [15]:
constant_features = constant_features["Var"].to_list()
print(constant_features)

['var_89', 'var_23', 'var_65', 'var_247', 'var_34', 'var_124', 'var_36', 'var_66', 'var_129', 'var_280', 'var_14', 'var_283', 'var_11', 'var_10', 'var_67', 'var_170', 'var_6', 'var_133', 'var_141', 'var_73', 'var_111', 'var_153', 'var_72', 'var_150', 'var_189', 'var_217', 'var_12', 'var_210', 'var_187', 'var_228', 'var_2', 'var_233', 'var_234', 'var_243', 'var_265', 'var_267', 'var_20', 'var_9', 'var_28', 'var_7', 'var_71', 'var_146', 'var_221', 'var_59', 'var_90', 'var_257', 'var_126', 'var_149', 'var_274', 'var_204', 'var_136', 'var_235', 'var_202', 'var_138', 'var_264', 'var_60', 'var_184', 'var_95', 'var_3', 'var_1', 'var_237', 'var_142', 'var_42', 'var_290', 'var_260', 'var_299', 'var_45', 'var_78', 'var_53', 'var_254', 'var_77', 'var_219', 'var_246', 'var_211', 'var_224', 'var_43', 'var_197', 'var_102', 'var_24', 'var_48', 'var_32', 'var_125', 'var_98', 'var_159', 'var_282', 'var_115', 'var_39', 'var_286', 'var_298', 'var_245', 'var_40', 'var_249', 'var_236', 'var_16', 'var_130',

In [16]:
print("Dataset shape before dropping constant and quasi constant features: ", df1.shape)
df1.drop(columns = constant_features, inplace=True)
print("Dataset shape after dropping constant and quasi constant features: ", df1.shape)

Dataset shape before dropping constant and quasi constant features:  (40000, 251)
Dataset shape after dropping constant and quasi constant features:  (40000, 119)


In [17]:
df1.head(10)

,var_4,var_8,var_15,var_17,var_18,var_21,var_29,var_30,var_31,var_35,...,var_276,var_277,var_279,var_281,var_284,var_288,var_292,var_295,var_300,target
33553,5.94,0,3,0.00,0.0,658.26,1.96,0.0,3,624.2208,...,0.0,0.0,2.94,0,0,0.0,0.0,0,0.0,0
9427,2.79,0,0,3.00,0.0,0.00,1.88,0.0,3,29100.0000,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0
199,0.00,0,0,0.00,0.0,0.00,1.82,0.0,3,0.0000,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0
12447,2.76,0,3,0.00,0.0,2.82,1.82,0.0,3,2.9100,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0
39489,2.73,0,3,2.82,0.0,2.97,1.84,0.0,3,2.7600,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0
42724,2.94,0,3,0.00,0.0,2.94,1.98,0.0,3,2.8500,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0
10822,0.00,0,0,0.00,0.0,0.00,1.92,0.0,3,0.0000,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0
49498,0.00,0,0,0.00,0.0,0.00,2.00,0.0,3,0.0000,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0
4144,0.00,0,0,0.00,0.0,0.00,1.96,0.0,3,0.0000,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0
36958,2.79,0,3,0.00,0.0,83.70,1.98,0.0,3,83.7000,...,0.0,0.0,0.00,0,0,0.0,0.0,0,0.0,0


In [18]:
from sklearn import datasets

cor_matrix = df1.corr().abs()
print(cor_matrix)

            var_4     var_8    var_15    var_17    var_18    var_21    var_29  \
var_4    1.000000  0.189747  0.498068  0.323940  0.143750  0.077032  0.035526   
var_8    0.189747  1.000000  0.035629  0.131467  0.100816  0.050483  0.001550   
var_15   0.498068  0.035629  1.000000  0.024371  0.074368  0.062503  0.039259   
var_17   0.323940  0.131467  0.024371  1.000000  0.051574  0.059792  0.004166   
var_18   0.143750  0.100816  0.074368  0.051574  1.000000  0.005640  0.002181   
...           ...       ...       ...       ...       ...       ...       ...   
var_288  0.204422  0.101648  0.009124  0.129711  0.025849  0.031032  0.004001   
var_292  0.122429  0.046945  0.016331  0.097005  0.015868  0.005846  0.003011   
var_295  0.186542  0.097976  0.067723  0.249892  0.010765  0.068702  0.008359   
var_300  0.091149  0.062666  0.050290  0.178499  0.003428  0.069702  0.004813   
target   0.080869  0.002699  0.132315  0.006617  0.003628  0.014267  0.001590   

           var_30    var_31

In [19]:
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
print(upper_tri)

         var_4     var_8    var_15    var_17    var_18    var_21    var_29  \
var_4      NaN  0.189747  0.498068  0.323940  0.143750  0.077032  0.035526   
var_8      NaN       NaN  0.035629  0.131467  0.100816  0.050483  0.001550   
var_15     NaN       NaN       NaN  0.024371  0.074368  0.062503  0.039259   
var_17     NaN       NaN       NaN       NaN  0.051574  0.059792  0.004166   
var_18     NaN       NaN       NaN       NaN       NaN  0.005640  0.002181   
...        ...       ...       ...       ...       ...       ...       ...   
var_288    NaN       NaN       NaN       NaN       NaN       NaN       NaN   
var_292    NaN       NaN       NaN       NaN       NaN       NaN       NaN   
var_295    NaN       NaN       NaN       NaN       NaN       NaN       NaN   
var_300    NaN       NaN       NaN       NaN       NaN       NaN       NaN   
target     NaN       NaN       NaN       NaN       NaN       NaN       NaN   

           var_30    var_31    var_35  ...   var_276   var_277 

C:\Users\Wojtek\AppData\Local\Temp\ipykernel_19564\12419535.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))


In [20]:
cols_to_drop = [column for column in upper_tri.columns if any(upper_tri[column]>0.95)]
print(cols_to_drop)

['var_64', 'var_88', 'var_100', 'var_123', 'var_143', 'var_155', 'var_160', 'var_162', 'var_163', 'var_164', 'var_166', 'var_179', 'var_186', 'var_190', 'var_191', 'var_203', 'var_222', 'var_252', 'var_253', 'var_255', 'var_258', 'var_259', 'var_261', 'var_262', 'var_270', 'var_272', 'var_277', 'var_281', 'var_295', 'var_300']


In [21]:
# dropping columns with high corellation
df1.drop(columns = list(cols_to_drop), axis=1, inplace=True)
print(df1.shape)

(40000, 89)


In [22]:
df1.head(10)

,var_4,var_8,var_15,var_17,var_18,var_21,var_29,var_30,var_31,var_35,...,var_244,var_266,var_271,var_275,var_276,var_279,var_284,var_288,var_292,target
33553,5.94,0,3,0.00,0.0,658.26,1.96,0.0,3,624.2208,...,2.85,0.0,0.0,0,0.0,2.94,0,0.0,0.0,0
9427,2.79,0,0,3.00,0.0,0.00,1.88,0.0,3,29100.0000,...,0.00,0.0,0.0,1,0.0,0.00,0,0.0,0.0,0
199,0.00,0,0,0.00,0.0,0.00,1.82,0.0,3,0.0000,...,0.00,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0
12447,2.76,0,3,0.00,0.0,2.82,1.82,0.0,3,2.9100,...,0.00,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0
39489,2.73,0,3,2.82,0.0,2.97,1.84,0.0,3,2.7600,...,0.00,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0
42724,2.94,0,3,0.00,0.0,2.94,1.98,0.0,3,2.8500,...,0.00,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0
10822,0.00,0,0,0.00,0.0,0.00,1.92,0.0,3,0.0000,...,0.00,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0
49498,0.00,0,0,0.00,0.0,0.00,2.00,0.0,3,0.0000,...,0.00,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0
4144,0.00,0,0,0.00,0.0,0.00,1.96,0.0,3,0.0000,...,0.00,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0
36958,2.79,0,3,0.00,0.0,83.70,1.98,0.0,3,83.7000,...,0.00,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0


In [23]:
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
cols_to_drop = [column for column in upper_tri.columns if any(upper_tri[column]<0.001)]
print(cols_to_drop)
# dropping columns with high corellation
df1.drop(columns = list(cols_to_drop), axis=1, inplace=True)
print(df1.shape)

# Columns with correlation less than 0.001, would we drop this?

['var_30', 'var_37', 'var_46', 'var_54', 'var_57', 'var_64', 'var_70', 'var_79', 'var_88', 'var_96', 'var_101', 'var_103', 'var_108', 'var_109', 'var_118', 'var_121', 'var_123', 'var_132', 'var_143', 'var_144', 'var_160', 'var_166', 'var_168', 'var_169', 'var_173', 'var_176', 'var_186', 'var_188', 'var_191', 'var_192', 'var_203', 'var_206', 'var_207', 'var_208', 'var_209', 'var_214', 'var_226', 'var_229', 'var_230', 'var_241', 'var_244', 'var_252', 'var_253', 'var_258', 'var_261', 'var_266', 'var_270', 'var_271', 'var_284', 'var_288', 'var_292', 'var_295', 'target']


C:\Users\Wojtek\AppData\Local\Temp\ipykernel_19564\3529097500.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))


KeyError: "['var_64', 'var_88', 'var_123', 'var_143', 'var_160', 'var_166', 'var_186', 'var_191', 'var_203', 'var_252', 'var_253', 'var_258', 'var_261', 'var_270', 'var_295'] not found in axis"

In [ ]:
print(df1.shape)

In [ ]:
cor_matrix = df1.corr().abs()

#should be graph with corellation matrix but too big data
import seaborn as sb
sb.heatmap(cor_matrix, cmap="Blues", annot=True)

Description:
 - For each column, it first computes the Z-score of each value in the column, relative to the column mean and standard deviation.

- It then takes the absolute Z-score because the direction does not matter, only if it is below the threshold.

- all(axis=1) ensures that for each row, all column satisfy the constraint.

- Finally, the result of this condition is used to index the dataframe.

In [ ]:
# description above
from scipy import stats

df1[(np.abs(stats.zscore(df1)) < 3).all(axis=1)]

In [ ]:
print(df1.shape)

In [ ]:
# Create a relative path to the directory you want to create
relative_path = "./preprocessed"

# Use os.makedirs to create the directory if it does not exist
if not os.path.exists(relative_path):
    os.makedirs(relative_path)
# exporting preprocessed dataset to csv
df1.to_csv('preprocessed/df1.csv', encoding='utf-8')